In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-11-07"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
16383,2024-11-07,Rússia Liga Vtb United,12:00,Samara,Saratov,1.42,2.90,153.5,1.90,1.90,-6.5,1.97,1.78,APjLmQyR,0.704225,0.344828,0.526316,0.526316,0.049053,187.812,131.118853,0.698139,1.2,1.643168,1.369306,112.42,2.600,2.392666,0.920256,13.0,496.934,839.809806,1.689983,1.2,1.643168,1.369306,94.16,6.702,11.354363,1.694175,-19.0,73,88,1.54,1.07,146.192,772.850,0.484499,0.000000,0.071653,-2.50,-0.500,-0.840000,0.652514,0.4,-0.252514,-3.45,-0.690,-2.753623,0.410088,0.5,0.089912
16384,2024-11-07,Lituânia Nkl,13:00,Suduva-Mantinga,Jurbarkas,1.59,2.15,165.5,1.87,1.79,-4.5,2.05,1.65,j3dRBcm3,0.628931,0.465116,0.534759,0.558659,0.094047,109.102,22.560399,0.206783,2.4,1.341641,0.559017,84.00,1.236,0.253634,0.205205,49.0,123.210,26.403772,0.214299,3.0,0.000000,0.000000,109.34,1.462,0.356118,0.243583,46.0,80,77,1.05,1.42,102.140,192.300,0.211754,0.030912,0.152888,-2.90,-0.580,-1.017241,0.839633,0.3,-0.539633,6.08,1.216,0.945724,0.472252,0.8,0.327748
16385,2024-11-07,Polônia 1. Liga,15:00,Trefl Sopot II,WKK Wroclaw,1.49,2.55,164.5,1.83,1.83,-6.5,1.87,1.80,db0T8NRk,0.671141,0.392157,0.546448,0.546448,0.063298,174.250,73.905321,0.424134,0.6,1.341641,2.236068,274.56,2.084,0.727757,0.349212,-28.0,166.098,40.875575,0.246093,1.2,1.643168,1.369306,133.77,2.214,0.605954,0.273692,-6.0,96,91,2.86,1.47,0.000,223.296,0.371056,0.000000,0.026974,0.00,0.000,inf,0.000000,0.0,0.000000,-3.36,-0.672,-2.306548,0.349195,0.4,0.050805
16386,2024-11-07,Polônia Liga De Basquete,13:30,Torun,Ostrow Wielkopolski,1.65,2.06,167.5,1.85,1.81,-3.5,1.99,1.69,IF56OADM,0.606061,0.485437,0.540541,0.552486,0.091497,233.418,96.109167,0.411747,1.2,1.643168,1.369306,255.96,3.178,1.448472,0.455781,-23.0,227.970,92.856728,0.407320,0.6,1.341641,2.236068,277.10,2.840,1.190735,0.419273,-15.0,79,85,3.24,3.26,171.706,225.630,0.156288,0.015456,0.115289,-1.70,-0.340,-1.911765,0.524506,0.5,-0.024506,-3.09,-0.618,-1.715210,0.512549,0.4,-0.112549
16387,2024-11-07,Nova Zelândia Tauihi,03:30,Mid-North Whai F,Mainland Pouakai F,1.59,2.15,160.5,1.86,1.80,-4.5,2.02,1.67,4jIJVeJG,0.628931,0.465116,0.537634,0.555556,0.094047,163.064,59.561911,0.365267,0.6,1.341641,2.236068,140.40,2.336,1.303468,0.557992,-35.0,205.302,94.422689,0.459921,1.8,1.643168,0.912871,327.52,2.580,0.789715,0.306091,-7.0,90,89,1.56,3.68,0.000,0.000,0.211754,0.023184,0.134139,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16463,2024-11-07,Dinamarca Liga De Basquete,15:00,Svendborg,Horsens,1.54,2.26,163.5,1.84,1.82,-4.5,1.96,1.72,vTxtc7Dj,0.649351,0.442478,0.543478,0.549451,0.091829,199.624,154.355496,0.773231,1.8,1.643168,0.912871,92.40,2.284,1.738658,0.761234,31.0,95.612,9.773491,0.102220,2.4,1.341641,0.559017,81.60,1.106,0.080187,0.072502,23.0,88,80,1.05,1.02,196.500,167.574,0.267956,0.007728,0.092231,-0.90,-0.180,-3.000000,0.618825,0.5,-0.118825,-0.40,-0.080,-15.750000,0.647909,0.7,0.052091
16464,2024-11-07,Dinamarca Liga De Basquete,14:30,Randers,Holbaek-Stenhus,1.01,9.35,174.5,1.86,1.80,-20.5,2.04,1.66,fe192TkT,0.990099,0.106952,0.537634,0.55

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
16387,03:30,Nova Zelândia Tauihi,Mid-North Whai F,Mainland Pouakai F,1.59,Back Home
16390,14:00,Lituânia Nkl,BC Olimpas Palanga,Vilkaviskis,1.56,Back Home
16416,22:30,Colômbia Lbp,Motilones del Norte,Cimarrones del Choco,1.56,Back Home
16430,16:45,Europa Euroliga,Paris,Partizan,1.87,Back Home
16443,23:00,México Lnbp,Diablos Rojos,Soles,1.58,Back Home
16467,23:15,México Lnbp,Fuerza Regia,Halcones de Xalapa,1.63,Back Home
